# Practical Exercise 6.03. Comparing fund performance with benchmark performance.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import empyrical as ep

# Define tickers for funds and benchmark
fund1_ticker = "IVV"   # iShares Core S&P 500 ETF
fund2_ticker = "VFINX" # Vanguard 500 Index Fund
benchmark_ticker = "^GSPC"  # S&P 500 Index

# Fetch historical price data
start_date = "2020-01-01"
end_date = "2025-01-01"

fund1_data = yf.download(fund1_ticker, start=start_date, end=end_date, auto_adjust=True, progress=False)['Close']
fund2_data = yf.download(fund2_ticker, start=start_date, end=end_date, auto_adjust=True, progress=False)['Close']
benchmark_data = yf.download(benchmark_ticker, start=start_date, auto_adjust=True, end=end_date, progress=False)['Close']

# Align data to avoid mismatches
fund1_data, benchmark_data = fund1_data.align(benchmark_data, join='inner', axis=0)
fund2_data, benchmark_data = fund2_data.align(benchmark_data, join='inner', axis=0)


# Compute daily returns (Ensure 1D pandas Series)
fund1_returns = fund1_data.pct_change().dropna().squeeze()
fund2_returns = fund2_data.pct_change().dropna().squeeze()
benchmark_returns = benchmark_data.pct_change().dropna().squeeze()

# --- EMPYRICAL CALCULATIONS FUNCTION ---
def compute_metrics_empyrical(fund_returns, benchmark_returns, risk_free_rate=0.02 / 252):
    # Ensure inputs are 1D pandas Series
    fund_returns = fund_returns.squeeze()
    benchmark_returns = benchmark_returns.squeeze()

    annualized_return = ep.annual_return(fund_returns)
    annualized_volatility = ep.annual_volatility(fund_returns)
    sharpe_ratio = ep.sharpe_ratio(fund_returns, risk_free=risk_free_rate)
    sortino_ratio = ep.sortino_ratio(fund_returns, required_return=risk_free_rate)
    max_drawdown = ep.max_drawdown(fund_returns)
    beta = ep.beta(fund_returns, benchmark_returns)
    alpha = ep.alpha(fund_returns, benchmark_returns, risk_free=risk_free_rate)
    calmar_ratio = ep.calmar_ratio(fund_returns)

    # Manually compute missing metrics
    tracking_error = (fund_returns - benchmark_returns).std() * np.sqrt(252)
    excess_return = ep.annual_return(fund_returns) - ep.annual_return(benchmark_returns)
    information_ratio = excess_return / tracking_error if tracking_error != 0 else np.nan

    return [annualized_return, annualized_volatility, sharpe_ratio, alpha, beta,
            tracking_error, information_ratio, sortino_ratio, max_drawdown, calmar_ratio]

# Compute metrics for both funds AND the S&P 500 index
metrics_labels = ["Annualized Return", "Annualized Volatility", "Sharpe Ratio", "Alpha (Annualized)", "Beta",
                  "Tracking Error", "Information Ratio", "Sortino Ratio", "Maximum Drawdown", "Calmar Ratio"]

fund1_metrics = compute_metrics_empyrical(fund1_returns, benchmark_returns)
fund2_metrics = compute_metrics_empyrical(fund2_returns, benchmark_returns)
sp500_metrics = compute_metrics_empyrical(benchmark_returns, benchmark_returns)  # S&P 500 self-comparison

# Create DataFrame for display
performance_metrics_df = pd.DataFrame({
    "Metric": metrics_labels,
    fund1_ticker: fund1_metrics,
    fund2_ticker: fund2_metrics,
    "S&P 500": sp500_metrics
})

# Display comparison table
print(performance_metrics_df)

# ========== PLOTTING REBASED PERFORMANCE ==========
plt.figure(figsize=(12, 6))

# Rebase all three series to 100
fund1_rebased = (fund1_data / fund1_data.iloc[0]) * 100
fund2_rebased = (fund2_data / fund2_data.iloc[0]) * 100
benchmark_rebased = (benchmark_data / benchmark_data.iloc[0]) * 100

# Plot Funds vs. Benchmark
plt.plot(fund1_rebased, label=fund1_ticker, color='blue', linewidth=2, alpha=0.7)
plt.plot(fund2_rebased, label=fund2_ticker, color='green', linewidth=2, alpha=0.7)
plt.plot(benchmark_rebased, label=benchmark_ticker, color='red', linestyle="dashed", linewidth=2, alpha=0.7)

# Formatting
plt.title("Rebased Performance (100 at Start)")
plt.xlabel("Date")
plt.ylabel("Index Value (Rebased to 100)")
plt.legend()
plt.grid()

# Show plot
plt.show()
